# FIFA Dataset Cleaning and Transformation

In [220]:
import numpy as np
import pandas as pd
import re

In [221]:
# Read the CSV file
df = pd.read_csv('fifa21_raw_data.csv')

/var/folders/gy/1f1fznt902v9q08yln7xz4vc0000gn/T/ipykernel_61346/3112989615.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fifa21_raw_data.csv')


## Data Examination

We need to examine all the columns and rows of the dataset in order to understand the data as well as to find where inconsisties occur. 


In [222]:
# How many columns are in the dataset? What are the names of them?
print(len(df.columns))
df.columns

77


Index(['photoUrl', 'LongName', 'playerUrl', 'Nationality', 'Positions', 'Name',
       'Age', '↓OVA', 'POT', 'Team & Contract', 'ID', 'Height', 'Weight',
       'foot', 'BOV', 'BP', 'Growth', 'Joined', 'Loan Date End', 'Value',
       'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 'PHY', 'Hits

In [223]:
df.head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,...,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,...,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,...,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,...,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,...,High,Medium,5 ★,91,85,86,94,36,59,\n273


In [224]:
df.tail()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
18974,https://cdn.sofifa.com/players/257/710/21_60.png,Mengxuan Zhang,http://sofifa.com/player/257710/mengxuan-zhang...,China PR,CB,Zhang Mengxuan,21,47,52,\n\n\n\nChongqing Dangdai Lifan FC SWM Team\n2...,...,Low,Low,1 ★,58,23,26,27,50,48,2
18975,https://cdn.sofifa.com/players/258/736/21_60.png,Vani Da Silva,http://sofifa.com/player/258736/vani-da-silva/...,England,ST,V. Da Silva,17,47,67,\n\n\n\nOldham Athletic\n2020 ~ 2021\n\n,...,Medium,Medium,1 ★,70,46,40,53,16,40,3
18976,https://cdn.sofifa.com/players/247/223/21_60.png,Ao Xia,http://sofifa.com/player/247223/ao-xia/210005/,China PR,CB,Xia Ao,21,47,55,\n\n\n\nWuhan Zall\n2018 ~ 2022\n\n,...,Medium,Medium,1 ★,64,28,26,38,48,51,3
18977,https://cdn.sofifa.com/players/258/760/21_60.png,Ben Hough,http://sofifa.com/player/258760/ben-hough/210005/,England,CM,B. Hough,17,47,67,\n\n\n\nOldham Athletic\n2020 ~ 2021\n\n,...,Medium,Medium,1 ★,64,40,48,49,35,45,5
18978,https://cdn.sofifa.com/players/255/958/21_60.png,Mateo Flores,http://sofifa.com/player/255958/mateo-flores/2...,Bolivia,CDM,M. Flores,19,47,63,\n\n\n\nClub Bolívar\n2020 ~ 2024\n\n,...,Medium,Medium,1 ★,57,32,43,48,44,49,2


We will look at all of the columns by using their column index positions.

In [225]:
# Alternative Way to Looking at all columns
column_inspec_indices = [*range(0,20)]
df.iloc[:, column_inspec_indices].head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,ID,Height,Weight,foot,BOV,BP,Growth,Joined,Loan Date End,Value
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,158023,"5'7""",159lbs,Left,93,RW,0,"Jul 1, 2004",NaN,€67.5M
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,20801,"6'2""",183lbs,Right,92,ST,0,"Jul 10, 2018",NaN,€46M
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,200389,"6'2""",192lbs,Right,91,GK,2,"Jul 16, 2014",NaN,€75M
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,192985,"5'11""",154lbs,Right,91,CAM,0,"Aug 30, 2015",NaN,€87M
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,190871,"5'9""",150lbs,Right,91,LW,0,"Aug 3, 2017",NaN,€90M


In [226]:
column_inspec_indices = [*range(20,40)]
df.iloc[:, column_inspec_indices].head()

,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance
0,€560K,€138.4M,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95
1,€220K,€75.9M,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71
2,€125K,€159.4M,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49
3,€370K,€161M,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76
4,€270K,€166.5M,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83


In [227]:
column_inspec_indices = [*range(40,60)]
df.iloc[:, column_inspec_indices].head()

,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling
0,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11
1,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11
2,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92
3,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13
4,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9


In [228]:
column_inspec_indices = [*range(60,77)]
df.iloc[:, column_inspec_indices].head()

,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,15,14,8,2231,466,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,15,14,11,2221,464,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77,\n344
2,78,90,90,1413,489,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,5,10,13,2304,485,5 ★,4★,High,High,4 ★,76,86,93,88,64,78,\n163
4,15,15,11,2175,451,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59,\n273


### General Observations

1. The `Team & Contract` column has newlines and needs to be cleaned up. Probably transformed into two separate columns 
2. The `Value`, `Wage`, `Release Clause` columns can be cleaned only to have the number.
3. The `W/F`, `SM`, and `IR` columns can be cleaned by removing the stars.
4. The `hits` column has a newline before the number of hits.
5. The `weight` column has lbs in all rows.

## Data Cleaning

### Step 1: `Team & Contract` Column

In [229]:
df['Team & Contract'].head()

0           \n\n\n\nFC Barcelona\n2004 ~ 2021\n\n
1               \n\n\n\nJuventus\n2018 ~ 2022\n\n
2        \n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n
3        \n\n\n\nManchester City\n2015 ~ 2023\n\n
4    \n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n
Name: Team & Contract, dtype: object

In [230]:
# Replace the '\n' with ' ' using the str.replace method in pandas
# Use rstrip and lstrip to remove leading and trailing spaces
df['Team & Contract'] = df['Team & Contract'].str.replace('\n',' ').str.lstrip().str.rstrip().str.replace('~', '-')
df['Team & Contract'].head()

0           FC Barcelona 2004 - 2021
1               Juventus 2018 - 2022
2        Atlético Madrid 2014 - 2023
3        Manchester City 2015 - 2023
4    Paris Saint-Germain 2017 - 2022
Name: Team & Contract, dtype: object

### Step 2: `Value`, `Wage`, `Release Clause` columns

In [231]:
# Change the column names to 'Value (€M)' from 'Value' and others using the rename method
df.rename(columns={'Value': 'Value (€M)', 'Wage': 'Wage (€K)', 'Release Clause': 'Release Clause (€M)'}, inplace=True)
new_columns_to_fix_1 = ['Value (€M)', 'Wage (€K)', 'Release Clause (€M)']
df[new_columns_to_fix_1].head()


,Value (€M),Wage (€K),Release Clause (€M)
0,€67.5M,€560K,€138.4M
1,€46M,€220K,€75.9M
2,€75M,€125K,€159.4M
3,€87M,€370K,€161M
4,€90M,€270K,€166.5M


In [232]:
# The regex expression extracts the numeric part, including decimals, from the columns
df['Value (€M)'] = df['Value (€M)'].str.extract(r'€([\d.]+)M').astype(float)
df['Wage (€K)'] = df['Wage (€K)'].str.extract(r'€([\d.]+)K').astype(float)
df['Release Clause (€M)'] = df['Release Clause (€M)'].str.extract(r'€([\d.]+)M').astype(float)
df[new_columns_to_fix_1].head()

,Value (€M),Wage (€K),Release Clause (€M)
0,67.5,560.0,138.4
1,46.0,220.0,75.9
2,75.0,125.0,159.4
3,87.0,370.0,161.0
4,90.0,270.0,166.5


### Step 3: `W/F`, `SM`, and `IR` columns

In [233]:
columns_to_fix_2 = ['W/F', 'SM', 'IR']
df[columns_to_fix_2].head()

,W/F,SM,IR
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [234]:
df.rename(columns={'W/F': 'W/F ★','SM': 'SM ★','IR': 'IR ★' }, inplace=True)
new_columns_to_fix_2 = ['W/F ★', 'SM ★', 'IR ★']
df[new_columns_to_fix_2].head()

,W/F ★,SM ★,IR ★
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [235]:
# We can replace the whole table of stars with the replace method
df = df.replace({'★': ''}, regex=True)
# Convert numbers to string to pd.to_numeric function (coerce parameter replaces anything not possible with NaN)
df[new_columns_to_fix_2] = df[new_columns_to_fix_2].apply(pd.to_numeric, errors='coerce')
df[new_columns_to_fix_2].head()

,W/F ★,SM ★,IR ★
0,4,4,5
1,4,5,5
2,3,1,3
3,5,4,4
4,5,5,5


### Step 4: `Hits` Column

In [236]:
df['Hits'].head()

0    \n372
1    \n344
2     \n86
3    \n163
4    \n273
Name: Hits, dtype: object

In [237]:
# Replace the `\n` with '' and then convert the hits column to numeric
df['Hits'] = df['Hits'].astype(str)
df['Hits'] = df['Hits'].str.replace('\n', '').apply(pd.to_numeric, errors='coerce').astype('Int64')
df['Hits'].head()

0    372
1    344
2     86
3    163
4    273
Name: Hits, dtype: Int64

### Step 5: `Weight` Column

In [238]:
df['Weight'].head()


0    159lbs
1    183lbs
2    192lbs
3    154lbs
4    150lbs
Name: Weight, dtype: object

In [239]:
df.rename(columns={'Weight': 'Weight (lbs)'}, inplace=True)
df['Weight (lbs)']=df['Weight (lbs)'].str.replace('lbs','').apply(pd.to_numeric, errors='coerce').astype('Int64')
df['Weight (lbs)'].head()

0    159
1    183
2    192
3    154
4    150
Name: Weight (lbs), dtype: Int64

### Final Check: Make sure certain columns are Numeric

We want to make sure that the wages and player statistics are numeric so that further data analysis can be conducted on these values.

In [240]:
numeric_columns = df.select_dtypes(include=['number']).columns
print(numeric_columns)

Index(['Age', '↓OVA', 'POT', 'ID', 'Weight (lbs)', 'BOV', 'Growth',
       'Value (€M)', 'Wage (€K)', 'Release Clause (€M)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F ★', 'SM ★', 'IR ★',
       'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits'],
      dtype='object')


We can now proceed to data transformation. 

## Data Transformation

1. Separate the height into feet and inches
2. Calculate the BMI of each player
3. Calculate the number of positions a player can play
4. Separate Team and Contract
5. Separate the joined column into year, month, and day columns.

### Step 1: Separate the `Height` column into feet and inches

In [241]:
df['Height'].head()

0     5'7"
1     6'2"
2     6'2"
3    5'11"
4     5'9"
Name: Height, dtype: object

In [242]:
# We will create two columns that has the feet and inches separately. 
df[['Height (ft)', 'Height (in)']] = df['Height'].str.extract(r"(\d+)\'(\d+)\"")
df[['Height (ft)', 'Height (in)']] = df[['Height (ft)', 'Height (in)']].apply(pd.to_numeric)
df[['Height', 'Height (ft)', 'Height (in)']].head()

,Height,Height (ft),Height (in)
0,"5'7""",5,7
1,"6'2""",6,2
2,"6'2""",6,2
3,"5'11""",5,11
4,"5'9""",5,9


### Step 2: Calculate the BMI for each player

We can create a new column using the following equation to solve for BMI: 

$$ BMI = \frac{{\text{{Weight (lbs)}}}}{{(\text{{Height (in)}})^2}} \times 703 $$


In [243]:
df['BMI'] = ((df['Weight (lbs)']) / ((df['Height (ft)'] * 12 + df['Height (in)']) ** 2)) * 703
df['BMI'].head()

0      24.9002
1    23.493243
2    24.648649
3    21.476294
4    22.148708
Name: BMI, dtype: Float64

### Step 3: Calculate the number of positions each player can play

In [244]:
df['Positions'].head()

0    RW ST CF
1       ST LW
2          GK
3      CAM CM
4      LW CAM
Name: Positions, dtype: object

In [245]:
# We could write a function that splits the string in each row into a list. The length of the list would be the number of positions.
def count_positions(player_pos):
    return len(player_pos.split(' '))
# Create a new column and use the apply function from pandas with the custom function
df['Num_Positions'] = df['Positions'].apply(count_positions)
df['Num_Positions'].head()


0    3
1    2
2    1
3    2
4    2
Name: Num_Positions, dtype: int64

### Step 4: Separate Team and Contract

In [246]:
df['Team & Contract'].head()

0           FC Barcelona 2004 - 2021
1               Juventus 2018 - 2022
2        Atlético Madrid 2014 - 2023
3        Manchester City 2015 - 2023
4    Paris Saint-Germain 2017 - 2022
Name: Team & Contract, dtype: object

In [247]:
# We can use the regex and the extract function to get the team and contract
df[['Team', 'Contract']] = df['Team & Contract'].str.extract(r'([^\d]+)\s*(\d+\s*-\s*\d+)')
df[['Team & Contract', 'Team', 'Contract']].head()

,Team & Contract,Team,Contract
0,FC Barcelona 2004 - 2021,FC Barcelona,2004 - 2021
1,Juventus 2018 - 2022,Juventus,2018 - 2022
2,Atlético Madrid 2014 - 2023,Atlético Madrid,2014 - 2023
3,Manchester City 2015 - 2023,Manchester City,2015 - 2023
4,Paris Saint-Germain 2017 - 2022,Paris Saint-Germain,2017 - 2022


### Step 5: Separate the `Joined` column into `Year`, `Month`, and `Day` columns

In [248]:
df['Joined'].head()

0     Jul 1, 2004
1    Jul 10, 2018
2    Jul 16, 2014
3    Aug 30, 2015
4     Aug 3, 2017
Name: Joined, dtype: object

In [249]:
# We can use extract and a regex pattern.
df[['Month', 'Day', 'Year']] = df['Joined'].str.extract(r'(\w{3}) (\d{1,2}), (\d{4})')
df[['Joined', 'Month', 'Day', 'Year']].head()

,Joined,Month,Day,Year
0,"Jul 1, 2004",Jul,1,2004
1,"Jul 10, 2018",Jul,10,2018
2,"Jul 16, 2014",Jul,16,2014
3,"Aug 30, 2015",Aug,30,2015
4,"Aug 3, 2017",Aug,3,2017


### Final Check

Now, we can inspect of all the columns and rows and make sure this a good dataset to proceed with to visualize. 

In [250]:
# How many columns are in the new cleaned and transformed data set?
print(len(df.columns))
df.columns

86


Index(['photoUrl', 'LongName', 'playerUrl', 'Nationality', 'Positions', 'Name',
       'Age', '↓OVA', 'POT', 'Team & Contract', 'ID', 'Height', 'Weight (lbs)',
       'foot', 'BOV', 'BP', 'Growth', 'Joined', 'Loan Date End', 'Value (€M)',
       'Wage (€K)', 'Release Clause (€M)', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'W/F ★', 'SM ★', 'A/W', 'D/W', 'IR ★',
       'PAC', 'SHO', 'PAS',

In [251]:
df.iloc[:, range(0,20)].head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,ID,Height,Weight (lbs),foot,BOV,BP,Growth,Joined,Loan Date End,Value (€M)
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,FC Barcelona 2004 - 2021,158023,"5'7""",159,Left,93,RW,0,"Jul 1, 2004",NaN,67.5
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,Juventus 2018 - 2022,20801,"6'2""",183,Right,92,ST,0,"Jul 10, 2018",NaN,46.0
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,Atlético Madrid 2014 - 2023,200389,"6'2""",192,Right,91,GK,2,"Jul 16, 2014",NaN,75.0
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,Manchester City 2015 - 2023,192985,"5'11""",154,Right,91,CAM,0,"Aug 30, 2015",NaN,87.0
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,Paris Saint-Germain 2017 - 2022,190871,"5'9""",150,Right,91,LW,0,"Aug 3, 2017",NaN,90.0


In [252]:
df.iloc[:, range(20,40)].head()

,Wage (€K),Release Clause (€M),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance
0,560.0,138.4,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95
1,220.0,75.9,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71
2,125.0,159.4,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49
3,370.0,161.0,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76
4,270.0,166.5,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83


In [253]:
df.iloc[:, range(40,60)].head()

,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling
0,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11
1,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11
2,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92
3,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13
4,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9


In [254]:
df.iloc[:, range(60,80)].head()

,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F ★,SM ★,A/W,D/W,IR ★,PAC,SHO,PAS,DRI,DEF,PHY,Hits,Height (ft),Height (in),BMI
0,15,14,8,2231,466,4,4,Medium,Low,5,85,92,91,95,38,65,372,5,7,24.9002
1,15,14,11,2221,464,4,5,High,Low,5,89,93,81,89,35,77,344,6,2,23.493243
2,78,90,90,1413,489,3,1,Medium,Medium,3,87,92,78,90,52,90,86,6,2,24.648649
3,5,10,13,2304,485,5,4,High,High,4,76,86,93,88,64,78,163,5,11,21.476294
4,15,15,11,2175,451,5,5,High,Medium,5,91,85,86,94,36,59,273,5,9,22.148708


In [255]:
df.iloc[:, range(80,86)].head()

,Num_Positions,Team,Contract,Month,Day,Year
0,3,FC Barcelona,2004 - 2021,Jul,1,2004
1,2,Juventus,2018 - 2022,Jul,10,2018
2,1,Atlético Madrid,2014 - 2023,Jul,16,2014
3,2,Manchester City,2015 - 2023,Aug,30,2015
4,2,Paris Saint-Germain,2017 - 2022,Aug,3,2017


In [256]:
# We can export this data in a csv file to the next notebook for data visualization.
df.to_csv('fifa21_cleaned_data.csv', index=False)
